### 実験可視化 精度計算 補正なし    

In [ ]:
# coding:utf-8
import json
import numpy as np
import pandas as pd
import ast
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import os
import glob
import seaborn as sns
import pickle
import math
from collections import OrderedDict

In [ ]:
# 解析基準位置に移動する関数
def move_root_dir(root_dir):
    while not (os.path.basename(os.getcwd())== root_dir):
        os.chdir('../')
    print('moving to:',os.path.basename(os.getcwd()))
    return os.getcwd()

### 入出力パス  

In [ ]:
work_code='20201012_preprocess_data'
root_dir = '2020_moox_morikoro_project'
root_path = move_root_dir(root_dir)
input_data_dir = 'input/20201012_raw_data'
save_csv_dir = 'preprocess_data/'+work_code+'/csvfile'
if not os.path.isdir(save_csv_dir):
    os.makedirs(save_csv_dir)
save_pkl_dir= 'preprocess_data/'+work_code+'/pkl'
if not os.path.isdir(save_pkl_dir):
    os.makedirs(save_pkl_dir)
save_img_dir= 'output/20201012_output_data/fig/'+work_code
if not os.path.isdir(save_img_dir):
    os.makedirs(save_img_dir)
save_static_dir = 'output/20201012_output_data/st_data/'+work_code
if not os.path.isdir(save_static_dir):
    os.makedirs(save_static_dir)

In [ ]:
# 関数ボックス
def read_csv_file(csv_file):
    df = pd.read_csv(csv_file)
    return df

def fill_data(df_data, st, fn):
    df = df_data.copy()
    df['Timestamp'] =pd.to_datetime(df['sensing_time'])
    df = df.set_index('Timestamp')
    df = df.resample('100ms').ffill()
    df = df.reset_index(drop=False)     
    df = df[(df['Timestamp'] > pd.to_datetime(st)) & (df['Timestamp'] <= pd.to_datetime(fn)) ]
    if(len(df)<1):
        return False, df_data        
    df = df.reset_index(drop=False)
    return True, df  

def get_data_75_3_body(df,i,head='data_body_'):
    axt = ['x', 'y', 'z']
    convert_list = [
        ['pelvis', 0],
        ['naval', 1],
        ['chest', 2],
        ['neck', 3],
        ['l_clavicle', 4],
        ['l_shoulder', 5],
        ['l_elbow', 6],
        ['l_wrist', 7],
        ['l_hand', 8],
        ['l_handtip', 9],
        ['l_thumb', 10],
        ['r_clavicle', 11],
        ['r_shoulder', 12],
        ['r_elbow', 13],
        ['r_wrist', 14],
        ['r_hand', 15],
        ['r_handtip', 16],
        ['r_thumb', 17],
        ['l_hip', 18],
        ['l_knee', 19],
        ['l_ankle', 20],
        ['l_foot', 21],
        ['r_hip', 22],
        ['r_knee', 23],
        ['r_ankle', 24],
        ['r_foot', 25],
        ['head', 26],
        ['nose', 27],
        ['l_eyes', 28],
        ['l_ear', 29],
        ['r_eyes', 30],
        ['r_ear', 31],
    ]
    body_dict={}
    body_dict['sensing_time']=df.at[i,'sensing_time']
    
    for cols in convert_list:
        body_dict[cols[0]]={}
        for ax in axt:
            col = head+cols[0]+'_'+ax
            body_dict[cols[0]][ax]=df.at[i,col]  
    return body_dict

def set_direction_data(df,i,data_dict,header):
    cols_list=[
        'face_direction_horizontal',
        'face_direction_vertical',
        'face_direction_bace_x','face_direction_bace_y','face_direction_bace_z',
        'face_direction_bace_right_eyes_x',
        'face_direction_bace_right_eyes_y',
        'face_direction_bace_right_eyes_z',
        'face_direction_bace_left_eyes_x',
        'face_direction_bace_left_eyes_y',
        'face_direction_bace_left_eyes_z'
    ]    
    for col in cols_list:
        colt =header+col
        df.at[i,colt]=data_dict[col]
    return df

def set_look_data(df,i,data_dict,header):
    cols_list=[
        'look_point_left_x','look_point_left_y','look_point_left_z',
        'look_point_left_object','look_point_left_distance',
        'look_point_left_eyes_x','look_point_left_eyes_y','look_point_left_eyes_z',  
        'look_point_left_eyes_object','look_point_left_eyes_distance',
        'look_point_center_x','look_point_center_y','look_point_center_z',          
        'look_point_center_object','look_point_center_distance',
        'look_point_right_eyes_x','look_point_right_eyes_y','look_point_right_eyes_z',
        'look_point_right_eyes_object','look_point_right_eyes_distance',
        'look_point_right_x','look_point_right_y','look_point_right_z',
        'look_point_right_object','look_point_right_distance',
        'look_point_top_x','look_point_top_y','look_point_top_z',
        'look_point_top_object','look_point_top_distance',
        'look_point_bottom_x','look_point_bottom_y','look_point_bottom_z',
        'look_point_bottom_object','look_point_bottom_distance',
    ]    
    for col in cols_list:
        colt =header+col
        df.at[i,colt]=data_dict[col]
    return df  

def calculate_data(df_data, offset_h=0.0, offset_v=0.0, head='data_body_'):
    # 補正
    look_points.set_offset(offset_h=offset_h, offset_v=offset_v)
    # 処理
    df = df_data.copy()    
    for i in range(len(df_data)):
        # 10 Hz 器官点データ取得
        body_dict = get_data_75_3_body(df, i, head=head)
        
        # 再計算(顔向き)
        face_direction.Calculate(body_dict, is_data=True)
        body_status_dict_r = face_direction.output_data
        df = set_direction_data(df,i,body_status_dict_r,'r_')
        
        # 再計算(視点)
        look_points.Calculate(body_status_dict_r, is_data=True)
        look_points_dict_r= look_points.output_data
        df = set_look_data(df,i,look_points_dict_r,'r_')
    return df

def side_block_9(xmax,xmin,ymax,ymin,zmax,zmin):
    box_9=[]

    xmax_=max([xmax,xmin])
    xmin_=min([xmax,xmin])
    xmax = xmax_
    xmin = xmin_
    ymax_=max([ymax,ymin])
    ymin_=min([ymax,ymin])
    ymax = ymax_
    ymin = ymin_
    zmax_=max([zmax,zmin])
    zmin_=min([zmax,zmin])
    zmax = zmax_
    zmin = zmin_    
    
    x03= xmin
    y03= ymin
    z03= zmin
    x13= xmin + int(abs(xmax-xmin)/3)
    y13= ymin + int(abs(ymax-ymin)/3)
    z13= zmin + int(abs(zmax-zmin)/3)
    x23= xmin + int(abs(xmax-xmin)*2/3)
    y23= ymin + int(abs(ymax-ymin)*2/3)
    z23= zmin + int(abs(zmax-zmin)*2/3)
    x33= xmax
    y33= ymax
    z33= zmax

    # 1
    cx= int((x23 + x33)/2.)
    cy= int((y23 + y33)/2.)
    cz= int((z23 + z33)/2.)
    box_9.append([cy,cy,cz, x23,y23,z23, x23,y33,z33, x33,y23,z23, x33,y33,z33, ])    
    # 2
    cx= int((x13 + x23)/2.)
    cy= int((y23 + y33)/2.)
    cz= int((z23 + z33)/2.)
    box_9.append([cy,cy,cz, x13,y23,z23, x13,y33,z33, x23,y23,z23, x23,y33,z33, ])
    # 3
    cx= int((x03 + x13)/2.)
    cy= int((y23 + y33)/2.)
    cz= int((z23 + z33)/2.)
    box_9.append([cy,cy,cz, x03,y23,z23, x03,y33,z33, x13,y23,z23, x13,y33,z33, ])    

    # 4
    cx= int((x23 + x33)/2.)
    cy= int((y13 + y23)/2.)
    cz= int((z13 + z23)/2.)
    box_9.append([cy,cy,cz, x23,y13,z13, x23,y23,z23, x33,y13,z13, x33,y23,z23, ])
    # 5
    cx= int((x13 + x23)/2.)
    cy= int((y13 + y23)/2.)
    cz= int((z13 + z23)/2.)
    box_9.append([cy,cy,cz, x13,y13,z13, x13,y23,z23, x23,y13,z13, x23,y23,z23, ])
    # 6
    cx= int((x03 + x13)/2.)
    cy= int((y13 + y23)/2.)
    cz= int((z13 + z23)/2.)    
    box_9.append([cy,cy,cz, x03,y13,z13, x03,y23,z23, x13,y13,z13, x13,y23,z23, ])    

    # 7
    cx= int((x23 + x33)/2.)
    cy= int((y03 + y13)/2.)
    cz= int((z03 + z13)/2.)
    box_9.append([cy,cy,cz, x23,y03,z03, x23,y13,z13, x33,y03,z03, x33,y13,z13, ])    
    # 8
    cx= int((x13 + x23)/2.)
    cy= int((y03 + y13)/2.)
    cz= int((z03 + z13)/2.)
    box_9.append([cy,cy,cz, x13,y03,z03, x13,y13,z13, x23,y03,z03, x23,y13,z13, ])
    # 9
    cx= int((x03 + x13)/2.)
    cy= int((y03 + y13)/2.)
    cz= int((z03 + z13)/2.)
    box_9.append([cy,cy,cz, x03,y03,z03, x03,y13,z13, x13,y03,z03, x13,y13,z13, ])

    return box_9

# 描画関数
def plot_object(plt,axis_x=0,axis_y=1,color='Glay'):
    object_tank = look_points.detect_cross.object_tank

    for object_box in object_tank:
        xbox=[]
        ybox=[]
        if(object_box[4][0]=='roof_body' or object_box[4][0]=='floor_body' or
          object_box[4][0]=='left_body' or #object_box[4][0]=='back_screen' or
          object_box[4][0]=='front_body' or object_box[4][0]=='rear_body' ):
            continue
        for i in range(4):
            box = object_box[i]
            xbox.append(box[axis_x])
            ybox.append(box[axis_y])        
            plt.scatter(box[axis_x],box[axis_y],marker='.',c='Gray',alpha=0.5)
        xmax = np.max(xbox)
        xmin = np.min(xbox)
        ymax = np.max(ybox)
        ymin = np.min(ybox)
        plt = frame_plot(plt, xmin=xmin, xmax=xmax,ymin=ymin,ymax=ymax,color="Gray",alpha=0.9)
        if(object_box[4][0]=='side_screen' or object_box[4][0]=='back_screen'):
            plt = cross_frame_plot_9(plt, xmin=xmin, xmax=xmax,ymin=ymin,ymax=ymax,fcolor="Gray",ccolor='Gray',alpha=0.5)
            
            
    return plt

def cross_frame_plot_9(plt,xmin=0, xmax=0,ymin=0,ymax=0,fcolor="Gray",ccolor='R',alpha=0.9):
    xmax_=max([xmax,xmin])
    xmin_=min([xmax,xmin])
    xmax = xmax_
    xmin = xmin_
    ymax_=max([ymax,ymin])
    ymin_=min([ymax,ymin])
    ymax = ymax_
    ymin = ymin_
    x03= xmin
    y03= ymin
    x13= xmin + int(abs(xmax-xmin)/3)
    y13= ymin + int(abs(ymax-ymin)/3)
    x23= xmin + int(abs(xmax-xmin)*2/3)
    y23= ymin + int(abs(ymax-ymin)*2/3)
    x33= xmax
    y33= ymax    
    # left line
    plt = cross_plot(plt, xmin=x03, xmax=x13,ymin=y03,ymax=y13,fcolor=fcolor,ccolor=ccolor,alpha=alpha)
    plt = cross_plot(plt, xmin=x03, xmax=x13,ymin=y13,ymax=y23,fcolor=fcolor,ccolor=ccolor,alpha=alpha)
    plt = cross_plot(plt, xmin=x03, xmax=x13,ymin=y23,ymax=y33,fcolor=fcolor,ccolor=ccolor,alpha=alpha)
    #print(x03,x13,x23,x33)
    #print(y03,y13,y23,y33)    
    # center line
    plt = cross_plot(plt, xmin=x13, xmax=x23,ymin=y03,ymax=y13,fcolor=fcolor,ccolor=ccolor,alpha=alpha)
    plt = cross_plot(plt, xmin=x13, xmax=x23,ymin=y13,ymax=y23,fcolor=fcolor,ccolor=ccolor,alpha=alpha)
    plt = cross_plot(plt, xmin=x13, xmax=x23,ymin=y23,ymax=y33,fcolor=fcolor,ccolor=ccolor,alpha=alpha)
    # right line
    plt = cross_plot(plt, xmin=x23, xmax=x33,ymin=y03,ymax=y13,fcolor=fcolor,ccolor=ccolor,alpha=alpha)
    plt = cross_plot(plt, xmin=x23, xmax=x33,ymin=y13,ymax=y23,fcolor=fcolor,ccolor=ccolor,alpha=alpha)
    plt = cross_plot(plt, xmin=x23, xmax=x33,ymin=y23,ymax=y33,fcolor=fcolor,ccolor=ccolor,alpha=alpha)
    return plt

def cross_frame_plot_4(plt,xmin=0, xmax=0,ymin=0,ymax=0,fcolor="Gray",ccolor='R',alpha=0.9):
    xc=(xmax+xmin)/2
    yc=(ymax+ymin)/2
    plt = cross_plot(plt, xmin=xmin, xmax=xc,ymin=ymin,ymax=yc,fcolor=fcolor,ccolor=ccolor,alpha=alpha)
    plt = cross_plot(plt, xmin=xmin, xmax=xc,ymin=yc,ymax=ymax,fcolor=fcolor,ccolor=ccolor,alpha=alpha)
    plt = cross_plot(plt, xmin=xc, xmax=xmax,ymin=ymin,ymax=yc,fcolor=fcolor,ccolor=ccolor,alpha=alpha)
    plt = cross_plot(plt, xmin=xc, xmax=xmax,ymin=yc,ymax=ymax,fcolor=fcolor,ccolor=ccolor,alpha=alpha)
    return plt

def cross_plot(plt,xmin=0, xmax=0,ymin=0,ymax=0,fcolor="Gray",ccolor='R',alpha=0.9):
    # cross
    plt.plot([xmax, xmin],[ymax, ymin], ccolor, alpha=alpha)
    plt.plot([xmin, xmax],[ymax, ymin], ccolor, alpha=alpha)
    # frame
    plt = frame_plot(plt, xmin=xmin, xmax=xmax,ymin=ymin,ymax=ymax,color=fcolor,alpha=alpha)
    return plt
    
def frame_plot(plt,xmin=0, xmax=0,ymin=0,ymax=0,color="Gray",alpha=0.9):
    # frame
    plt.plot([xmin, xmin],[ymin, ymax], c=color, alpha=alpha)
    plt.plot([xmin, xmax],[ymin, ymin], c=color, alpha=alpha)
    plt.plot([xmax, xmax],[ymin, ymax], c=color, alpha=alpha)
    plt.plot([xmin, xmax],[ymax, ymax], c=color, alpha=alpha)    
    return plt

def get_pallet():
    cmap = plt.get_cmap("tab10")
    cmap = [cmap(0)[0:3],cmap(1)[0:3],cmap(2)[0:3],cmap(3)[0:3],cmap(4)[0:3],cmap(5)[0:3],
            cmap(6)[0:3],cmap(7)[0:3],cmap(8)[0:3],cmap(9)[0:3],cmap(10)[0:3],
            cmap(11)[0:3],cmap(12)[0:3],cmap(13)[0:3],cmap(14)[0:3],cmap(15)[0:3],
           ]
    cbox=[]
    for c in cmap:
        color='#'+format(int(c[0]*255), '02x')+format(int(c[1]*255), '02x')+format(int(c[2]*255), '02x')
        cbox.append(color)
    cmap = cbox
    return cmap

def get_true_pos(tbox,ids):
    dx=75.75/2.0
    dy=74.22/2.0
    target_lt_x = tbox[3] 
    target_lt_y = tbox[4]
    target_lt_z = tbox[5]
    target_lb_x = tbox[6] 
    target_lb_y = tbox[7]
    target_lb_z = tbox[8]
    target_rt_x = tbox[9] 
    target_rt_y = tbox[10]
    target_rt_z = tbox[11]
    target_rb_x = tbox[12] 
    target_rb_y = tbox[13]
    target_rb_z = tbox[14]
    target_id=ids
    if(target_id==5):
        true_x=(target_lt_x+target_rb_x)/2
        true_y=(target_lt_y+target_rb_y)/2
    elif(target_id==1):
        true_x=target_rt_x - dx
        true_y=target_lb_y - dy
    elif(target_id==2):
        true_x=(target_lt_x+target_rb_x)/2
        true_y=target_rb_y -dy
    elif(target_id==3):
        true_x=target_lb_x + dx
        true_y=target_lb_y - dy
    elif(target_id==4):
        true_x=target_rt_x - dx
        true_y=(target_lt_y+target_rb_y)/2
    elif(target_id==6):
        true_x=target_lt_x + dx
        true_y=(target_lt_y+target_rb_y)/2
    elif(target_id==7):
        true_x=target_rb_x -dx
        true_y=target_lt_y + dy
    elif(target_id==8):
        true_x=(target_lb_x+target_rb_x)/2
        true_y=target_rt_y + dy
    elif(target_id==9):
        true_x=target_lb_x +dx
        true_y=target_rt_y + dy
    else:
        true_x=(target_lt_x+target_rb_x)/2
        true_y=(target_lt_y+target_rb_y)/2 
    true_z=(target_lt_z+target_rb_z)/2
    return true_x,true_y,true_z

def get_convert_parameter(df,colx,coly):           
    # 修正演算
    x1=[0,0]
    x2=[0,0]
    x3=[0,0]
    x4=[0,0]
    cx1=[0,0]
    cx2=[0,0]
    cx3=[0,0]
    cx4=[0,0]

    for ids in range(1,10):
        df = df_data.copy()  
        c=cmap[ids]
        df=df[df['stable_gaze_flag']==ids]            
        df=df.reset_index(drop=True)
        #print(ids,len(df))
        if(ids==0):
            continue
        tbox=side_target_box[ids-1]
        true_x,true_y,true_z = get_true_pos(tbox,ids)
        if(ids==3):
            x1=[np.nanmean(df[colx]),np.nanmean(df[coly])]
            #plt.scatter(x1[0],x1[1],label='mean_3',color="K", marker="X")
            cx1=[true_x,true_y]
            #plt.scatter(true_x,true_y,label='true_3',color="K", marker="X")
            #print(true_x,true_y,'3')
        elif(ids==9):
            x2=[np.nanmean(df[colx]),np.nanmean(df[coly])]
            #plt.scatter(x2[0],x2[1],label='mean_9',color="K", marker="X")
            cx2=[true_x,true_y]
            #plt.scatter(true_x,true_y,label='true_9',color="K", marker="X")
            #print(true_x,true_y,'9')
        elif(ids==7):
            x3=[np.nanmean(df[colx]),np.nanmean(df[coly])]
            #plt.scatter(x3[0],x3[1],label='mean_7',color="K", marker="X")
            cx3=[true_x,true_y]
            #plt.scatter(true_x,true_y,label='true_7',color="K", marker="X")
            #print(true_x,true_y,'7')
        elif(ids==1):
            x4=[np.nanmean(df[colx]),np.nanmean(df[coly])]
            #plt.scatter(x4[0],x4[1],label='mean_1',color="K", marker="X")
            cx4=[true_x,true_y]
            #plt.scatter(true_x,true_y,label='true_1',color="K", marker="X")
            #print(true_x,true_y,'1')

        # 可視化
        #x=df[colx]
        #y=df[coly]
        #plt.scatter(x,y,label=str(ids),color=c,alpha=0.1)

    origin_corner_list=[x4,x3,x2,x1]
    destination_corner_list=[cx4,cx3,cx2,cx1]
    homography_matrix = get_trans_met(origin_corner_list,destination_corner_list)
    #print(origin_corner_list)
    #print(destination_corner_list)
    #print(homography_matrix)
    return homography_matrix,origin_corner_list,destination_corner_list

section_sep=[[0,1050,1],[1050,1950,2],[1950,2750,3],[2750,3560,4],[3560,3990,5],[3990,4460,6],]


In [ ]:
def __convert_corner_list_to_homography_param(point_list):
    # type: (np.ndarray) -> np.ndarray
    """
    点の集合（dim * n の配列）を同次座標系に変換する
    :param point_list: ホモグラフィ行列変換用に整形を行う座標リスト
    :return: ホモグラフィ変換用行列
    """
    return np.vstack((point_list, np.ones((1, point_list.shape[1]))))

def __normalize(point_list):
    # type: (np.ndarray) -> (np.ndarray, np.ndarray)
    """
    正規化処理
    :param point_list: 正規化対象の座標リスト
    :return: 正規化結果, 正規化に用いた係数行列(理解が怪しい)
    """
    m = np.mean(point_list[:2], axis=1)
    max_std = max(np.std(point_list[:2], axis=1)) + 1e-9
    c = np.diag([1 / max_std, 1 / max_std, 1])
    c[0][2] = -m[0] / max_std
    c[1][2] = -m[1] / max_std
    return np.dot(c, point_list), c

def calculate_homography_matrix(origin, dest):
    # type: (np.ndarray, np.ndarray) -> np.ndarray
    """
    線形DLT法にて、 変換元を変換先に対応づけるホモグラフィ行列を求める。先行実装に倣う。
    :param origin: ホモグラフィ行列計算用の初期座標配列
    :param dest: ホモグラフィ行列計算用の移動先座標配列
    :return: 計算結果のホモグラフィ行列(3 x 3)
    """
    assert origin.shape == dest.shape

    origin = __convert_corner_list_to_homography_param(origin.T)
    dest = __convert_corner_list_to_homography_param(dest.T)

    # 点を調整する（数値計算上重要）
    origin, c1 = __normalize(origin)  # 変換元
    dest, c2 = __normalize(dest)      # 変換先
    # 線形法計算のための行列を作る。
    nbr_correspondences = origin.shape[1]
    a = np.zeros((2 * nbr_correspondences, 9))
    for i in range(nbr_correspondences):
        a[2 * i] = [-origin[0][i], -origin[1][i], -1, 0, 0, 0, dest[0][i] * origin[0][i], dest[0][i] * origin[1][i],
                    dest[0][i]]
        a[2 * i + 1] = [0, 0, 0, -origin[0][i], -origin[1][i], -1, dest[1][i] * origin[0][i], dest[1][i] * origin[1][i],
                        dest[1][i]]
    u, s, v = np.linalg.svd(a)
    homography_matrix = v[8].reshape((3, 3))
    homography_matrix = np.dot(np.linalg.inv(c2), np.dot(homography_matrix, c1))
    homography_matrix = homography_matrix / homography_matrix[2, 2]
    return homography_matrix

# 射影変換
def get_trans_met(origin_corner_list,destination_corner_list):
    """
        :param img: 元画像
        :param origin_corner_list: 切り出し対象物の4角の座標リスト [[x1, y1], [x2, y2], [x3, y3], [x4, y4]]
        :param destination_corner_list: 対象物の移動先となるの4角の座標リスト
        :return: 変換後の画像
    """
    # origin_corner_list=[[1,1],[1,0],[0,0],[0,1]]
    # destination_corner_list=[[2,2],[2,0],[0,0],[0,2]]
    # ホモグラフィ変換に必要な切り出し座標・変換先座標のパラメータを生成。
    origin_corner_list = np.array(origin_corner_list)
    destination_corner_list = np.array(destination_corner_list)

    # ホモグラフィ変換行列(変換先 -> 変換元 の変換用)を取得
    # いずれの切り出し座標にも通じる、変換後の座標との関係を示す数式(行列)を求めるイメージ。
    homography_matrix = calculate_homography_matrix(origin_corner_list, destination_corner_list)
    homography_param = homography_matrix.ravel()
    
    inv_homography_matrix = np.linalg.inv(homography_matrix)
    inv_homography_matrix /= inv_homography_matrix[2, 2]
    inv_homography_param = inv_homography_matrix.ravel()
        
    return homography_param

def trans_point(homography_param, point):
    x=point[0]
    y=point[1]
    a=homography_param[0]
    b=homography_param[1]
    c=homography_param[2]
    d=homography_param[3]
    e=homography_param[4]
    f=homography_param[5]
    g=homography_param[6]
    h=homography_param[7]
    x0=(a*x+b*y+c)/(g*x+h*y+1)
    y0=(d*x+e*y+f)/(g*x+h*y+1)
    convert_point=[x0,y0] 
    return convert_point



In [ ]:
# 射影変換 sample 動作検証
origin_corner_list=[[1,1],[1,0],[0,0],[0,1]]
destination_corner_list=[[2,2],[2,-2],[-2,-2],[-2,2]]
homography_param = get_trans_met(origin_corner_list,destination_corner_list)
print(homography_param,len(homography_param))

for point in origin_corner_list:
    convert_point = trans_point(homography_param, point)
    print(point, ' > ',convert_point)

print()
origin_corner_list=[[-1,-1],[0,1],[1,1],[1,0]]
destination_corner_list=[[2,2],[2,-2],[-2,-2],[-2,2]]
homography_param = get_trans_met(origin_corner_list,destination_corner_list)
print(homography_param,len(homography_param))

for point in origin_corner_list:
    convert_point = trans_point(homography_param, point)
    print(point, ' > ',convert_point)

print()

origin_corner_list=[[1,1],[0,1],[-1,-1],[1,0]]
destination_corner_list=[[2,2],[2,-2],[-2,-2],[-2,2]]
homography_param = get_trans_met(origin_corner_list,destination_corner_list)
print(homography_param,len(homography_param))

for point in origin_corner_list:
    convert_point = trans_point(homography_param, point)
    print(point, ' > ',convert_point)

### データ読み込み  

In [ ]:
# # データセット作成
# exp_code = "02_20201012_exp"

# csv_files=glob.glob(save_csv_dir+'/data_df_rx_r02/*.csv')

# data_box=[]
# #for t_box in time_box:
# for csv_file in csv_files:
#     df_data=pd.read_csv(csv_file)
#     filename=os.path.basename(csv_file)
#     pp=filename[7:10]
#     ss=filename[10:13]
#     tt=filename[-7:-4]
#     st=''
#     fn=''
#     mask=int(filename[-9:-8])
#     print(pp,ss,tt,mask)
#     box=[pp,ss,tt,st,fn,df_data,mask]
#     data_box.append(box)
# print('data_count: ',len(data_box))
# finename = save_pkl_dir+'/'+exp_code+'.pkl'
# with open(finename, 'wb') as web:
#     pickle.dump(data_box, web)
# print('saved ', finename)

# finename = save_pkl_dir+'/'+exp_code+'.pkl'
# with open(finename, 'rb') as web:
#       data_box = pickle.load(web)
#       #print techacademy

In [ ]:
# アノテーション済みデータ読み込み
exp_code = "02_20201012_exp"
finename = save_pkl_dir+'/'+exp_code+'.pkl'
with open(finename, 'rb') as web:
      data_box = pickle.load(web)
      #print techacademy
print(data_box[0][0:5],data_box[0][6:7],len(data_box[0]))

In [ ]:
# セクション区切りフレーム数のリスト
section_sep=[[0,1050,1],[1050,1950,2],[1950,2750,3],[2750,3560,4],[3560,3990,5],[3990,4460,6],]

In [ ]:
# 位置補正　無補正
# 領域設定
xmax=max([514,-698])
xmin=min([514,-698])
ymax=max([163,847])
ymin=min([163,847])
zmax=max([733,757])
zmin=min([733,757])
side_target_box=side_block_9(xmax,xmin,ymax,ymin,zmax,zmin)

# 代表点算出
exp_code = "03_20201012_exp_section_prefix"
save_dir = save_img_dir+'/'+exp_code
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

for alg in ['r02',]:
    print(alg)
    for box in data_box:
        pp=box[0]
        ss=box[1]
        tt=box[2]
        st=box[3]
        fn=box[4]
        df_data=box[5]
        is_mask=int(box[6])
        # データ確認
        df_data_raw = df_data.copy()
        # 10色パレット
        cmap = get_pallet()

        # 時刻
        for [st,fn,no] in section_sep:
            df_data=df_data_raw.copy()
            print(st,fn,no)
            df_data=df_data[st:fn]
            df_data=df_data.reset_index(drop=True)
            
            if(no>4):
                continue

            # 絶対
            plt.figure(figsize=(14,8))
            ax = plt.axes()

            # 格納
            calced_st_box=[]
            calced_col_box=[]
            # side視点分布
            title=exp_code+'_'+pp+'_'+ss+'_'+tt+'_side_look_mask_'+str(is_mask)+'_'+alg+'_'+'section_'+str(no)
            colx=alg+'_look_point_center_x'
            coly=alg+'_look_point_center_y'

            # 本計算
            # look vecter
            for ids in range(1,10):
                df = df_data.copy()  
                c=cmap[ids]
                df=df[df['stable_gaze_flag']==ids] 
                #print(ids,len(df))
                df=df.reset_index(drop=True)
                if(ids==0):
                    continue
                tbox=side_target_box[ids-1]
                target_lt_x = tbox[3] 
                target_lt_y = tbox[4]
                target_lt_z = tbox[5]
                target_lb_x = tbox[6] 
                target_lb_y = tbox[7]
                target_lb_z = tbox[8]
                target_rt_x = tbox[9] 
                target_rt_y = tbox[10]
                target_rt_z = tbox[11]
                target_rb_x = tbox[12] 
                target_rb_y = tbox[13]
                target_rb_z = tbox[14]
                true_x,true_y,true_z = get_true_pos(tbox,ids)

                r=111
                draw_circle = plt.Circle((true_x, true_y), r ,fill=False, color=c, alpha=0.8)
                plt.gcf().gca().add_artist(draw_circle)
                plt.plot([target_lt_x, target_lb_x],[target_lt_y, target_lb_y], c, alpha=0.2)
                plt.plot([target_lb_x, target_rb_x],[target_lb_y, target_rb_y], c, alpha=0.2)
                plt.plot([target_rb_x, target_rt_x],[target_rb_y, target_rt_y], c, alpha=0.2)
                plt.plot([target_rt_x, target_lt_x],[target_rt_y, target_lt_y], c, alpha=0.2)

                x=df[colx]
                y=df[coly]
                plt.scatter(x,y,label=str(ids),color=c,alpha=0.4)
            

            # 精度計算
            for ids in range(1,10):
                df = df_data.copy()  
                c=cmap[ids]
                df=df[df['stable_gaze_flag']==ids]
                if(len(df)<1):
                    #print('pass',ids)
                    continue
                #print(ids,len(df))
                df=df.reset_index(drop=True)
                if(ids==0):
                    continue
                target_rt_y = tbox[9] 
                target_rb_x = tbox[12] 
                target_rb_y = tbox[13] 
                
                tbox=side_target_box[ids-1]
                true_x,true_y,true_z = get_true_pos(tbox,ids)
                # 統計
                header=alg+'_'
                tr=ids
                st_box=[pp,ss,tt,is_mask,tr]
                col_box=['pp','ss','tt','mask','target']

                # 統計計算
                std_x=df[header+'look_point_center_x'].std()
                std_y=df[header+'look_point_center_y'].std()
                #print('STD X:{:.2f} STD Y:{:.2f}'.format(std_x,std_y))
                st_box.extend([std_x,std_y])
                col_box.extend(['STD_X [mm]','STD_Y [mm]'])
                # dir_error
                header=alg+'_'
                for i in range(len(df)):
                    if(alg=='data_status_look_points'):
                        header2='data_status_face_direction_'
                    else:
                        header2=header
                    bace_point_x = df.at[i,header2+'face_direction_bace_x']
                    bace_point_y = df.at[i,header2+'face_direction_bace_y']
                    bace_point_z = df.at[i,header2+'face_direction_bace_z']
                    bace_point = np.array([bace_point_x, bace_point_y, bace_point_z,])

                    look_point_x = df.at[i,header+'look_point_center_x']
                    look_point_y = df.at[i,header+'look_point_center_y']
                    look_point_z = df.at[i,header+'look_point_center_z']
                    look_point = np.array([look_point_x, look_point_y, look_point_z,])

                    look_true_x = true_x
                    look_true_y = true_y
                    look_true_z = true_z
                    look_true = np.array([look_true_x, look_true_y, look_true_z,])
                    # print(bace_point, look_points, look_true)

                    # 正しい注視点の角度を算出
                    # 鉛直
                    zpoint=(zmax+zmin)/2.
                    L = abs(bace_point[2] - zpoint )
                    l = (look_true[1] - bace_point[1])
                    true_v_dir = np.rad2deg(np.arctan2(l,L))

                    L = abs(bace_point[2] - zpoint )
                    l = (look_point[1] - bace_point[1])
                    v_dir = np.rad2deg(np.arctan2(l,L))

                    # 水平
                    L = abs(bace_point[2] - zpoint )
                    l = (look_true[0] - bace_point[0])
                    true_h_dir = np.rad2deg(np.arctan2(l,L))

                    L = abs(bace_point[2] - zpoint )
                    l = (look_point[0] - bace_point[0])
                    h_dir = np.rad2deg(np.arctan2(l,L))
                    # 距離アノマリーを算出
                    v_anomaly = look_point[1] - look_true[1]
                    h_anomaly = look_point[0] - look_true[0]
                    df.at[i,'v_error'] = v_anomaly
                    df.at[i,'h_error'] = h_anomaly
                    # 角度アノマリーを算出
                    v_anomaly_dir = v_dir - true_v_dir
                    h_anomaly_dir = h_dir - true_h_dir
                    df.at[i,'v_error_dir'] = v_anomaly_dir
                    df.at[i,'h_error_dir'] = h_anomaly_dir
                    df.at[i,'error_dir']= np.sqrt(v_anomaly_dir*v_anomaly_dir + h_anomaly_dir*h_anomaly_dir)
                st_box.extend([
                    np.nanmean(df['h_error']),
                    np.nanmean(df['v_error']),
                    np.nanmean(df['h_error_dir']),
                    np.nanmean(df['v_error_dir']),
                ])
                col_box.extend(['h_error[mm]','v_error[mm]',
                                       'h_error[deg]','v_error[deg]'])
                df['abs_h_error_dir']=np.abs(df['h_error_dir'])
                df['abs_v_error_dir']=np.abs(df['v_error_dir'])
                st_box.extend([
                    df['abs_h_error_dir'].mean(),
                    df['abs_v_error_dir'].mean(),
                    df['error_dir'].mean(),
                ])
                col_box.extend(['MAE_h[deg]','MAE_v[deg]','MAE[deg]'])
                
                # 注視するよう教示点周囲にある注視点の数
                up_deg=5
                dn_deg=-5
                df['in_deg_xy']=0
                cole='error_dir'
                df.loc[(df[cole] > dn_deg) &(df[cole] < up_deg),'in_deg_xy'] = 1
                in_deg_xy = 100*df['in_deg_xy'].sum()/len(df)
                st_box.extend([in_deg_xy,'{}/{}'.format(df['in_deg_xy'].sum(),len(df))])
                col_box.extend(['in_deg_xy[%]','in_deg_xy'])            

                df['in_data_xy']=0
                if(target_rt_x  < target_lt_x):
                    df.loc[(df[colx] > target_rt_x) &(df[colx] < target_lt_x),'in_data_x'] = 1
                else:
                    df.loc[(df[colx] < target_rt_x) &(df[colx] > target_lt_x),'in_data_x'] = 1
                st_box.extend([100*df['in_data_x'].sum()/len(df),'{}/{}'.format(df['in_data_x'].sum(),len(df))])
                col_box.extend(['in_data_x[%]','in_data_x'])

                df['in_data_y']=0
                if(target_lb_y  < target_lt_y):
                    df.loc[(df[coly] > target_lb_y )&(df[coly] < target_lt_y),'in_data_y'] = 1
                else:
                    df.loc[(df[coly] < target_lb_y )&(df[coly] > target_lt_y),'in_data_y'] = 1
                st_box.extend([100*df['in_data_y'].sum()/len(df),'{}/{}'.format(df['in_data_y'].sum(),len(df))])
                col_box.extend(['in_data_y[%]','in_data_y'])

                df['in_data_xy']=0
                df.loc[(df['in_data_x'] ==1 )&(df['in_data_y'] ==1),'in_data_xy'] = 1
                in_data_xy = 100*df['in_data_xy'].sum()/len(df)
                st_box.extend([100*df['in_data_xy'].sum()/len(df),'{}/{}'.format(df['in_data_xy'].sum(),len(df))])
                col_box.extend(['in_data_xy[%]','in_data_xy'])                
                #print(col_box)
                #print(st_box)
                pcode = '{:.2f} [%]'.format(in_deg_xy)
                plt.text(true_x, true_y, pcode, fontsize=15,rotation=0 )

                # 保存
                calced_st_box.append(st_box)
                calced_col_box=col_box
            df_st=pd.DataFrame(calced_st_box,columns=calced_col_box)

            # 
            save_dir2=save_static_dir+'/'+exp_code
            if not os.path.isdir(save_dir2):
                os.makedirs(save_dir2)
            #print(df_st)
            save_file=save_dir2+'/'+title+'_stat.csv'
            df_st.to_csv(save_file,index=False)
            print(save_file)            
            
            plt.title(title)
            plt.xlim(1000,-1800)
            plt.ylim(-100,1500)
            plt.xlabel(' x [mm]')
            plt.ylabel(' y [mm]')
            handles, labels = plt.gca().get_legend_handles_labels()
            by_label = OrderedDict(zip(labels, handles))
            plt.legend(by_label.values(), by_label.keys(),loc="upper left",frameon=False)
            # save as png
            save_file=title+'.png'
            save_path=os.path.join(save_dir, save_file)
            plt.savefig(save_path)
            plt.show()  